In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

with open('../temp_files/RoadSegCapacLane_dict_JSON.json', 'r') as json_file:
    RoadSegCapacLane_dict_JSON = json.load(json_file)

In [3]:
##### read in raw data

import openpyxl

data_folder = '/home/jzh/Dropbox/Research/\
Data-driven_estimation_inverse_optimization/INRIX/Raw_data/'

# load filtered INRIX attribute table raw data
wb_INRIX = openpyxl.load_workbook(data_folder + 'filtered_INRIX_attribute_table_ext.xlsx')

# load lookup table raw data
wb_lookup = openpyxl.load_workbook(data_folder + 'roadinv_id_to_tmc_lookup.xlsx') 

# get sheet name from workbook
sheet_INRIX_name = wb_INRIX.sheetnames[0].encode('utf-8')
sheet_lookup_name = wb_lookup.sheetnames[0].encode('utf-8')

# get sheet of filtered INRIX attribute table raw data
sheet_INRIX = wb_INRIX.get_sheet_by_name(sheet_INRIX_name)

# get sheet of lookup table raw data
sheet_lookup = wb_lookup.get_sheet_by_name(sheet_lookup_name)

In [4]:
##### extract attributes of interest from INRIX sheet

tmc_list = []
road_num_list = []
shape_length_list = []

for i in xrange(2, 1 + sheet_INRIX.max_row):
    tmc_list.append(sheet_INRIX.cell(row=i, column=2).value.encode('utf-8'))
    road_num_list.append(sheet_INRIX.cell(row=i, column=3).value.encode('utf-8'))
    shape_length_list.append(sheet_INRIX.cell(row=i, column=13).value)

assert(len(tmc_list) == len(road_num_list) and \
       len(road_num_list) == len(shape_length_list)) 

In [5]:
##### extract attributes of interest from lookup sheet

road_inv_ID_lookup_list = []
tmc_lookup_list = []

for i in xrange(2, 1 + sheet_lookup.max_row):
    road_inv_ID_lookup_list.append(sheet_lookup.cell(row=i, column=1).value) 
    tmc_lookup_list.append(str(sheet_lookup.cell(row=i, column=4).value))

assert(len(road_inv_ID_lookup_list) == len(tmc_lookup_list))    

# instantiation of LookUp class
look_up = LookUp(road_inv_ID_lookup_list, tmc_lookup_list)

# make a dictionary from look_up
tmc_roadInv_dict = {i:j for (i, j) in zip(look_up.tmc, look_up.road_inv_ID)}

In [6]:
capac_AM = []
capac_MD = [] 
capac_PM = [] 
capac_NT = []

for tmc in tmc_list:
    if not(tmc in tmc_roadInv_dict.keys()):
        capac_AM.append(-1.0)
        capac_MD.append(-1.0)
        capac_PM.append(-1.0)
        capac_NT.append(-1.0)
    else:
        road_inv_ID = str(tmc_roadInv_dict[tmc])
        if not(road_inv_ID in RoadSegCapacLane_dict_JSON.keys()):
            capac_AM.append(-1.0)
            capac_MD.append(-1.0)
            capac_PM.append(-1.0)
            capac_NT.append(-1.0)
        else:
            capac_AM.append(RoadSegCapacLane_dict_JSON[road_inv_ID]['AB_AM_capac'])
            capac_MD.append(RoadSegCapacLane_dict_JSON[road_inv_ID]['AB_MD_capac'])
            capac_PM.append(RoadSegCapacLane_dict_JSON[road_inv_ID]['AB_PM_capac'])
            capac_NT.append(RoadSegCapacLane_dict_JSON[road_inv_ID]['AB_NT_capac'])

In [7]:
# instantiation of RoadSegInrCapac class
road_seg_inr_capac = RoadSegInrCapac(tmc_list, road_num_list, shape_length_list, \
                                     capac_AM, capac_MD, capac_PM, capac_NT)

zdump(road_seg_inr_capac, '../temp_files/road_seg_inr_capac_ext.pkz')